In [ ]:
!pip install -U datasets

In [ ]:
import pandas as pd
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EvalPrediction,
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
import random
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
print("device:", device)

In [ ]:
train_path = Path("data/training_split.csv")
val_path = Path("data/validation_split.csv")

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)

LABEL2ID = {"negative": 0, "neutral": 1, "positive": 2}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

train_df["label"] = train_df["label"].map(LABEL2ID)
val_df["label"] = val_df["label"].map(LABEL2ID)

assert train_df["label"].isna().sum() == 0
assert val_df["label"].isna().sum() == 0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", force_download=True)
train_ds = Dataset.from_pandas(train_df[["sentence", "label"]])
val_ds = Dataset.from_pandas(val_df[["sentence", "label"]])

def tokenize_function(example):
    return tokenizer(
        example["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)

train_ds = train_ds.remove_columns(["sentence"])
val_ds = val_ds.remove_columns(["sentence"])

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3,
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

def compute_metrics(eval_pred: EvalPrediction):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    preds  = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}


arguments = dict(
    output_dir="./distilbert_baseline",
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    seed=seed,
    fp16=True,
    dataloader_num_workers=4
)

training_args = TrainingArguments(**arguments)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print('training...')
trainer.train()

print('evaluating...')
trainer.evaluate()